In [1]:
import pandas as pd
import glob, os
import re
import itertools
from collections import Counter

def read_file_by_lines(file):
    try:
        with open(file, 'r', encoding='latin-1') as f:
            lines = f.read().splitlines()
        return lines
    except:
        with open(file, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines()
        return lines
    
    
def extract_date(string):
    # Extract the value using regular expressions
    match = re.search(r'(\d{6})_(\d+)', string)

    if match:
        extracted_value = match.group(2)
        return(int(extracted_value))
    
def extract_time(string):
    # Extract the value using regular expressions
    match = re.search(r'_(\d+)_', string)

    if match:
        extracted_value = match.group(1)
        return(extracted_value)
    
def extract_timestamp(time):
    
    # Extract hour, minute, and second using regular expressions
    match = re.search(r"\[(\d{3}):(\d{2}):(\d{2});((\d{3})|(\d{2}))\]", time)

    if match:
        hour = match.group(1)
        minute = match.group(2)
        second = match.group(3)
        millisecond = match.group(4)
        
        return [int(hour), int(minute), int(second), int(millisecond)]
    else:
        print(time)

def get_frag_in_elapse_time(df, based_col, new_col, gap_time=35):
    # Assuming you have a DataFrame named 'df' with appropriate columns

    last_sent = 0
    next_sent = 0
    elapse_time0 = 0
#     gap_time = 60
    
    df[new_col] = pd.Series(dtype='int')

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        frag_count_gt1 = row[based_col]
        time_span = row['time_span']

        next_sent = index
        if frag_count_gt1 > 0:
            # Set indicator = 1 and update last_sent
            df.at[index, new_col] = 1
            last_sent = index
            elapse_time0 = 0
        else:
            if last_sent == 0:
                # Set indicator = 0 if no matches found yet
                df.at[index, new_col] = 0
            else:

                elapse_time0 += time_span

                if elapse_time0 <= gap_time:
                    # Set indicator = 1 if time span <= gap_time
                    df.at[index, new_col] = 1
                else:
                    # Set indicator = 0 for rows between last_sent+1 and next_sent-1
                    df.loc[last_sent + 1:next_sent, new_col] = 0
                    elapse_time0 = 0
                    last_sent=0

def CC_process(cc_files_dic, transcript_file, output_file_path, date):
    print(transcript_file)
#     set and extract start time of 9am
    if date[-3:-2] == '6' or date[-3:-2] == '7':
        Nine_AM_time = 130000
    else:
        Nine_AM_time = 140000
        
    
    df_transcript = pd.read_excel(transcript_file, index_col=0)  
    
    df_transcript = df_transcript[df_transcript.columns.drop(list(df_transcript.filter(regex='Unnamed')))].reset_index()
    
    with pd.ExcelWriter(output_file_path) as writer:
    
        for news_type, cc_files in cc_files_dic.items():

            target_ls = []

            for file in glob.glob(cc_files):
                target_ls.append(file)
                
            if target_ls != []:

                df = pd.DataFrame(target_ls, columns=['File Names'])

                df['clip_starttime'] = df['File Names'].apply(extract_date)
                df['DATE'] = df['File Names'].apply(extract_time)
                df['raw_input'] = df.apply(lambda x: read_file_by_lines(x['File Names']), axis=1)
                df = df.explode('raw_input').reset_index(drop=True)
                df[['hour', 'minute', 'second', 'millisecond']] = df.apply(lambda x: extract_timestamp(x['raw_input']), axis=1, result_type="expand")
                df = df.sort_values(['clip_starttime', 'hour', 'minute', 'second', 'millisecond']).reset_index(drop=True)
                
                df['raw_input'] = df['raw_input'].apply(lambda x: x.split(']')[1].strip('>- ')).fillna('')
                df['number_of_words'] = df['raw_input'].apply(lambda x: len(re.sub(r'[^\w\s]', '', x).split(' ')))
                df['cleaned_text'] = df['raw_input'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x).replace(" ", "")))
                df['real_time_clip_seconds'] = df['hour']*3600+df['minute']*60+df['second']+(df['clip_starttime']-Nine_AM_time)/10000*3600+32400
                df_transcript['cleaned_sentence'] = df_transcript['sentence'].fillna('').apply(lambda x: re.sub(r'[^\w\s]', '', str(x).replace(" ", "")))
                df_transcript['trans_timestamp_seconds'] = df_transcript['time_hour_part']*3600+df_transcript['time_min_part']*60+df_transcript['time_sec_part']
                df['occurrence_sent_num'] = df.apply(lambda x: [df_transcript['sentence_num'][idx] for idx, sent in enumerate(df_transcript['cleaned_sentence'].values) if x['cleaned_text'].lower() in sent.lower()] if x['number_of_words']>1 else '' , axis=1)
                df['timed_matches'] = df.apply(lambda x: [sent_num for (sent_num, timed_match) in \
                                                  zip(x.occurrence_sent_num ,[Time_tol > abs(x.real_time_clip_seconds - \
                                                df_transcript['trans_timestamp_seconds'][df_transcript['sentence_num'].tolist().index(i)]) \
                                                for i in x['occurrence_sent_num']]) if timed_match], axis=1)

                df.to_excel(writer, sheet_name=news_type + '_Closed_Captioning')

                occurrence_dic = Counter(list(itertools.chain.from_iterable(df[df['cleaned_text'].str.len()>1]['timed_matches'].values)))

                df_transcript['Frag_count_gt_'+news_type] = df_transcript['sentence_num'].apply(lambda x: occurrence_dic[x] if x in occurrence_dic else 0)

                get_frag_in_elapse_time(df_transcript, 'Frag_count_gt_'+news_type, 'Frag_in_elasp_time_'+news_type)

                # Add two dummy columns
                df_transcript['dummy_preceding'+news_type] = 0
                df_transcript['dummy_subsequent'+news_type] = 0

                # Iterate over each row in the DataFrame
                for index, row in df_transcript.iterrows():
                    # Calculate the sum of the preceding 5 sentences
                    if index > 4:
                        preceding_sum = df_transcript['Frag_in_elasp_time_'+news_type][index - 5:index].sum()
                    else:
                        preceding_sum = df_transcript['Frag_in_elasp_time_'+news_type][0:index].sum()

                    # Calculate the sum of the subsequent 5 sentences or till the end of the file
                    if index < len(df) - 5:
                        subsequent_sum = df_transcript['Frag_in_elasp_time_'+news_type][index + 1:index + 6].sum()
                    else:
                        subsequent_sum = df_transcript['Frag_in_elasp_time_'+news_type][index + 1:].sum()

                    # Set dummy variables
                    df_transcript.at[index, 'dummy_preceding'+news_type] = 1 if preceding_sum > 0 else 0
                    df_transcript.at[index, 'dummy_subsequent'+news_type] = 1 if subsequent_sum > 0 else 0

                # Set the values in the third column based on the conditions
                df_transcript['Frag_range5'+news_type] = 0

                for index, row in df_transcript.iterrows():
                    second_value = row['Frag_in_elasp_time_'+news_type]
                    dummy_preceding = row['dummy_preceding'+news_type]
                    dummy_subsequent = row['dummy_subsequent'+news_type]

                    if second_value == 0:
                        df_transcript.at[index, 'Frag_range5'+news_type] = 0
                    elif second_value == 1 and dummy_preceding == 0 and dummy_subsequent == 0:
                        df_transcript.at[index, 'Frag_range5'+news_type] = 0
                    else:
                        df_transcript.at[index, 'Frag_range5'+news_type] = 1

                # Drop the dummy columns if they are no longer needed
                df_transcript = df_transcript.drop(['dummy_preceding'+news_type, 'dummy_subsequent'+news_type], axis=1)
            
        df_transcript.to_excel(writer, sheet_name='Transcript')
        

Time_tol = 100



C:\Users\czhao\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\czhao\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\czhao\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# this is how to process all files
all_folders = glob.glob("0_ChenZhao/*")
dates = [i[-8:] for i in all_folders]
transcript_files = glob.glob("interuption_transcriptsv4/*")
transcript_date = [i[-19:-11] for i in glob.glob("interuption_transcriptsv4/*")]
target_files = [transcript_files[transcript_date.index(date)] for date in dates]

In [3]:
# cases for testing one output
all_folders = ['0_ChenZhao/cc5_20100224']
target_files = ['interuption_transcriptsv4/Bernanke_20100224_v10_3_with_part2.xlsx']
dates = ['20100224']

In [10]:
for folder, transcript, date in zip(all_folders,target_files, dates):
    cc_files_dic = {"CNBC": "{}/CNBC*.txt".format(folder), "BLOOMBERG": "{}/BLOOMBERG*.txt".format(folder), "FBC": "{}/FBC*.txt".format(folder)}

    CC_process(cc_files_dic, transcript, 'output_folder/CC_output_' + date + '.xlsx', date)

interuption_transcriptsv4/Bernanke_20100224_v10_3_with_part2.xlsx
